# ARTS AMSR2 multi-FOV surface simulations

In [1]:
#  d duncan, 28/01/19
#  adapting a mixture of arts and matlab scripts from patrick to ipynb
#   so that i can play around with running the forward model before doing
#   any surface retrievals 

# this setup, in words:
### - define an antenna response that is frequency dependent, gaussian, symmetric
### - define sensor position and line of sight for each scan at middle of scan
### - define an angular grid (zenith & azimuth) for each scan that encompasses npix
### - sensor angles are all 'absolute' whereas the angular grid and antenna response 
###    are all relative to the central bore sight of the scan
### - a lat/lon grid is also defined that must encompass all observation points simulated
### - the same angular grid and simulation setup is assumed/copied for all nscans, and 
###    mblock_dlos_grid allows all simulations to be run at the same time

### *** this setup outputs simulated TBs for chosen channels of AMSR2
###    with antenna patterns taken into account and decoupled from any retrieval grid, with
###    pencil beam calculations sampling the antenna pattern according to the angular grid
###    defined

In [2]:

###nfoot    =  7  # currently both nscans and npix (assuming a square setup)
npix     =  8  # pixels across each scan considered
nscans   =  5  # consecutive scans considered
angfac   =  5   # angular samples per degree
npol     =  2   # number of polarizations considered (2=V/H both, 1=intensity only)

In [3]:

%env ARTS_INCLUDE_PATH=/home/dudavid/arts/controlfiles/
%env ARTS_BUILD_PATH=/home/dudavid/arts/build/
%env ARTS_DATA_PATH=/home/dudavid/arts/arts-xml/ 

%matplotlib inline
#from h5py import File

from typhon.arts.workspace import Workspace, arts_agenda
ws = Workspace(verbosity=0)
ws.execute_controlfile("general/general.arts")
ws.execute_controlfile("general/continua.arts")
ws.execute_controlfile("general/agendas.arts")
ws.execute_controlfile("general/planet_earth.arts")

from typhon.arts.workspace.variables import *

ws.Copy( ws.abs_xsec_agenda, ws.abs_xsec_agenda__noCIA )
ws.Copy( ws.iy_main_agenda, ws.iy_main_agenda__Emission )
ws.Copy( ws.iy_space_agenda, ws.iy_space_agenda__CosmicBackground )
ws.Copy( ws.propmat_clearsky_agenda, ws.propmat_clearsky_agenda__OnTheFly )
ws.Copy( ws.ppath_agenda, ws.ppath_agenda__FollowSensorLosPath )
ws.Copy( ws.ppath_step_agenda, ws.ppath_step_agenda__GeometricPath )
@arts_agenda
def geo_pos_agendaPY(ws):
    ws.geo_posEndOfPpath()
ws.Copy( ws.geo_pos_agenda, geo_pos_agendaPY)  ## new one, set by patrick in his cfile

# define absorbing species and sensor (here using metmm library, used again below)
ws.abs_speciesSet(species=["H2O-PWR98","O2-PWR93","N2-SelfContStandardType"])#,"liquidcloud-ELL07"])
ws.abs_lines_per_speciesSetEmpty()

ws.stokes_dim = npol # to get V and H pol set to 2
ws.iy_unit = "PlanckBT" # equivalent: ws.StringSet( iy_unit, "PlanckBT" )

env: ARTS_INCLUDE_PATH=/home/dudavid/arts/controlfiles/
env: ARTS_BUILD_PATH=/home/dudavid/arts/build/
env: ARTS_DATA_PATH=/home/dudavid/arts/arts-xml/
Loading ARTS API from: /home/dudavid/arts/build/src/libarts_api.so


In [4]:
# Define lat/lon grid that observation points and angular grid lie within
lat0     = 0.62  # center of grid in lat, lon
lon0     = 0
nlat, nlon = 30, 30    # number of lat and lon divisions
latwid, lonwid = 10, 10  # just asssume square for now
 
lat_grid = lat0 + np.linspace(-latwid,latwid,nlat) # [-3:0.25:3]
lon_grid = lon0 + np.linspace(-lonwid,lonwid,nlat) # [-3:0.25:3]
 
ws.lat_grid = lat_grid
ws.lon_grid = lon_grid

In [5]:
# set up atmosphere and surface variables:
ws.atmosphere_dim = 3 
p = np.array([1015.,950.,900.,800.,650.,500.,300.,100.])*100.0  # keep it simple
#p = np.array([1015.,975.,950.,925.,900.,850.,800.,750.,700.,650.,600.,550.,500.,400.,300.,200.,100.])*100.0
ws.p_grid = p[:] 
ws.AtmRawRead( basename = "planets/Earth/Fascod/tropical/tropical") #tropical atmosphere assumed
ws.AtmosphereSet3D()
ws.AtmFieldsCalcExpand1D()  # set to given p_grid or z_grid

In [6]:
# Surface properties

s_names = ['Water skin temperature','Wind speed','Wind direction','Salinity']
s_data = np.zeros([len(s_names), lat_grid.size, lon_grid.size])

tmean                     = 296   # sst mean
wsp                       = 7.3   # 10m wind speed
s_data[0,:,:] = tmean
s_data[0,2,4] = tmean+3.5 # some preturbation
s_data[1,:,:] = wsp
s_data[1,5,1] = wsp-3.2 # some perturbation
s_data[2,:,:] = 90 # token wind direction value
s_data[3,:,:] = 0.034 # token salinity value
# write these to ARTS variables (don't write to xml)
ws.Copy(ws.surface_props_names, s_names)
ws.Copy(ws.surface_props_data, s_data)
ws.MatrixSetConstant(ws.z_surface, nlat, nlon, 0.0)


In [7]:
# Set and calculate some basic variables for antenna and scanning
### all will be replaced or superseded when moving to using L1R data as input
 
zsat   = 699.7e3      # Satellite altitde [m] -- can read this in later
vsat   = 6.76e3       # Satellite velocity [m/s]
dt     = 2.6e-3       # Integration time [s] -- 2.6ms for low freqs, 1.3ms for 89GHz
rpm    = 40           # Rotations per minute -- same for AMSR-E and AMSR2
 
# find nautical mile constant?  1852m = 1nmi
m2deg  = 1/(60*1852) #constants('NAUTICAL_MILE'))    # Conversion from m to latitude
dang   = dt * 360 * rpm / 60                  # Angular distance between samples
dlat   = m2deg * vsat * 60 / rpm              # Latitude distance between scans
print(dang,dlat)

0.624 0.09125269978401726


In [9]:
# define antenna pattern:

# for a given angular resolution (might vary), width=2 yields 99.96% of total power @6GHz 
#   compared to width=20 and 99.99999% @10GHz, width=1.5 yields 99.0 and 99.994.
width = 2.0            # Max half-width of antenna  (HPBW is 1.8deg, so awidth=2 covers over 2x the HPBW)
resol  = 0.02          # angular resolution (same in zenith/azimuth for now)
#resol, width = dang/(angfac*5), awidth # set here instead of calling func
print('angular resolution of response grid [deg]: ',resol)
adata = [[ 6.925e9 ,  1.8],   # AMSR2 center frequency and beamwidth (deg)
         [ 7.300e9 ,  1.8],   # assumed the same for V/H polarizations
         [ 10.65e9 ,  1.2]] #,
         #[ 18.70e9 ,  0.65],
         #[ 23.80e9 ,  0.75],
         #[ 36.50e9 ,  0.35],
         #[ 89.00e9 ,  0.15] ]
adata = np.array(adata).transpose()

# define zenith, azimuth grid on the ground (relative to bore sight) -- assumed to be 
x  = np.arange( -width, width+resol, resol )  #this gives angular antenna response points in za,aa
x2 = x**2   # since assumed origin is 0, do squaring here 
nf, nx = len(adata[0,:]), np.size(x)

from typhon.arts.griddedfield import GriddedField4
gf4 = GriddedField4()
gf4.name = 'AMSR2 antenna response'
gf4.gridnames =  [ 'Polarisation', 'Frequency', 'Zenith angle', 'Azimuth angle' ]
###  note: za & aa are equally spaced, and we're treating V/H as having identical responses
if npol==2: gsp="1" 
else: gsp="0"
gf4.grids = [ [gsp], adata[0,:], x, x ]
gf4.dataname  = 'Response'
gf4.data      = np.zeros([ 1, nf, nx, nx ])
print('size of antenna_reponse grid: ',gf4.data.shape)

#blah = [4589.014643600229,
#4589.014643600229,
#2039.5620638223247]
for i in range(nf):
    si = adata[1,i] / (2*np.sqrt(2*np.log(2)))  # calculate standard deviation first, based on HPBW
    gf4.data[0,i,:,:] = np.exp( - np.tile(x2,[nx,1])/si**2 - np.tile(x2,[nx,1]).transpose()/ si**2 )
    #gf4.data[0,i,:,:] = np.exp( - (np.tile(x2,[nx,1]) + np.tile(x2,[nx,1]).transpose())/ si**2 ) # equivalent
    #print(np.sum(gf4.data[0,i,:,:])/blah[i])



angular resolution of response grid [deg]:  0.02
size of antenna_reponse grid:  (1, 3, 201, 201)


In [10]:
# frequency grid of simulation is defined according to sensor setup above (may change with use of metmm)
f_grid = np.copy(gf4.grids[1])  # array with each frequency (not channel)

ws.f_grid.value = f_grid
print('f_grid: ',ws.f_grid.value)
 

f_grid:  [6.925e+09 7.300e+09 1.065e+10]


In [11]:
# Determine bore-sight angles to use for one scan 
 
# steps defines the pixels in one scan
psteps   = np.arange(-(npix-1)/2, (npix-1)/2 +1, 1)  #  assumes npix is odd, with bore in the middle
ssteps   = np.arange(-(nscans-1)/2, (nscans-1)/2 +1, 1) # assumes nscans is odd, with bore in the middle
#print('pixel steps',psteps)

#  center line of sight... defined as AMSR2 EIA and 0 azimuth angle
los0    = [ 180-47.5, 0 ]  # 47.5 is off-nadir angle of AMSR2

# bsights defines the bore sight zenith and azimuth angles across the scan 
##-- just for one scan, assumed to be same for each scan?
bsights = np.array([ np.repeat(los0[0],npix), los0[1]+dang*psteps ]).transpose()  # size: [npix,2]
print('bsights:',bsights)

bsights: [[132.5    -2.184]
 [132.5    -1.56 ]
 [132.5    -0.936]
 [132.5    -0.312]
 [132.5     0.312]
 [132.5     0.936]
 [132.5     1.56 ]
 [132.5     2.184]]


In [12]:
# Store data describing angles

#n = np.floor( angfac * width / dang )  # number per degree * angular width / distance between boresights
n = np.floor( width * angfac )  # angular width * angular samples per deg
print(n)
#za_grid = np.array( los0[0] + (dang/angfac) * np.arange(-n,n+1))  # zenith angle grid
za_grid = np.array( los0[0] +  np.arange(-n,n+1)/angfac)  ## effectively taking 'width' on either side of 0
print(za_grid) # should be something like 

en = np.floor((bsights[-1,1]+width)*angfac) ## yields az from boresight at edge plus 'width' (could prob do width/2)
#en       = 1 + np.floor( angfac * ( bsights[-1,1] + width ) / dang ) # azimuth of boresight at edge +width
print(en)
#aa_grid = np.array( los0[1] + (dang/angfac) * np.arange(-en,en+1))  # azimuth angle grid
aa_grid = np.array( los0[1] + np.arange(-en,en+1/angfac)/angfac)  # azimuth angle grid
print(aa_grid)




10.0
[130.5 130.7 130.9 131.1 131.3 131.5 131.7 131.9 132.1 132.3 132.5 132.7
 132.9 133.1 133.3 133.5 133.7 133.9 134.1 134.3 134.5]
20.0
[-4.  -3.8 -3.6 -3.4 -3.2 -3.  -2.8 -2.6 -2.4 -2.2 -2.  -1.8 -1.6 -1.4
 -1.2 -1.  -0.8 -0.6 -0.4 -0.2  0.   0.2  0.4  0.6  0.8  1.   1.2  1.4
  1.6  1.8  2.   2.2  2.4  2.6  2.8  3.   3.2  3.4  3.6  3.8  4. ]


In [13]:
# Set sensor_pos, sensor_los

ws.sensor_los = np.tile(los0, [nscans,1])  # sensor line of sight for each scan

shift   = 7.5    # Latitude shift to centre calculations around lat0

#### sensor_pos should be columns of altitude, SClat, SClon  ###  
####   so take all this from L1R files and transform X/Y/Z to lat/lon
# currently one alt/sclat/sclon for each scan (i.e. each of nscans)
ws.sensor_pos = np.hstack([np.repeat(zsat, nscans).reshape(nscans,1),
                          np.array(lat0 - shift + dlat * ssteps).reshape(nscans,1), 
                          np.repeat(lon0, nscans).reshape(nscans,1)])

print(ws.sensor_pos.value)
#print(np.transpose(ws.sensor_pos.value))
print(np.shape(ws.sensor_pos.value))

#  spos = [ repmat(zsat,nfoot,1), lat0-shift+dlat*steps, repmat(lon0,nfoot,1) ];
#xmlStore( fullfile(wfolder,'sensor_pos.xml'), spos, 'Matrix' )

[[ 6.9970000e+05 -7.0625054e+00  0.0000000e+00]
 [ 6.9970000e+05 -6.9712527e+00  0.0000000e+00]
 [ 6.9970000e+05 -6.8800000e+00  0.0000000e+00]
 [ 6.9970000e+05 -6.7887473e+00  0.0000000e+00]
 [ 6.9970000e+05 -6.6974946e+00  0.0000000e+00]]
(5, 3)


In [14]:
# other controlfile things done in common.arts
ws.jacobianOff()
ws.cloudboxOff()

In [15]:
# define surface agenda:
varnam = ["Optical depth"]
ws.Copy(ws.iy_aux_vars, varnam)
ws.VectorCreate( "transmittance" )
ws.transmittance = np.ones( ws.f_grid.value.shape ) * 0.7
@arts_agenda
def iy_surface_agendaPY(ws):
    ws.specular_losCalc()
    # if ppathCalc is called and uses assumed inputs, problems! so specify all!
    ws.ppathCalc(ws.ppath_agenda, ws.ppath_lmax, 
       ws.ppath_lraytrace, ws.atmgeom_checked, ws.t_field, ws.z_field, 
       ws.vmr_field, ws.f_grid, ws.cloudbox_on, ws.cloudbox_checked, 
       ws.ppath_inside_cloudbox_do, ws.rtp_pos, ws.specular_los, ws.rte_pos2 )
    ws.iyEmissionStandard()
    
    ws.transmittanceFromIy_aux(transmittance=ws.transmittance)
    ws.SurfaceFastem( transmittance = ws.transmittance, fastem_version=6 ) 
    ws.iySurfaceRtpropCalc()
    
ws.Copy(ws.iy_surface_agenda, iy_surface_agendaPY) # copy python-defined agenda to ARTS

In [16]:
# perform some checks:
ws.abs_xsec_agenda_checkedCalc()
ws.propmat_clearsky_agenda_checkedCalc()
ws.atmfields_checkedCalc( bad_partition_functions_ok = 1 )
ws.atmgeom_checkedCalc()
ws.cloudbox_checkedCalc()

In [17]:
# create vector mblock_reference_los and matrix mblock_target_los (to be used by DiffZaAa)
mblock_reference_los = np.array(los0)
print(mblock_reference_los)

# want to have all individual angle pairs distinct... so repeat one serially and one n times
mblock_target_los = np.array(np.transpose( np.stack( [ np.repeat(za_grid, aa_grid.size) , 
                                           np.tile(  aa_grid, za_grid.size) ]) ) , order = "C")
## need to specify 'C' order or else stride is wrong and array values are read wrong by ARTS

print(mblock_target_los[0:10,:])
print(mblock_target_los.shape)

#### this causes kernel to die, throws arts error "Assertion `za != 0 && za != 180' failed."
# which exists in 'ppath.cc' -- none of the *za* values going are zeros, but if column 2 has nonzeros, it works?

### 31/01 -- mblock_target_los is now correct, but still this function has weird bug... and output looks wrong!

ws.DiffZaAa(ws.mblock_dlos_grid, mblock_reference_los, mblock_target_los)

print(np.shape(ws.mblock_dlos_grid.value))
print(ws.mblock_dlos_grid.value[0:10,:])
print(info(ws.mblock_dlos_grid.value[:,0]),info(ws.mblock_dlos_grid.value[:,1]))

[132.5   0. ]
[[130.5  -4. ]
 [130.5  -3.8]
 [130.5  -3.6]
 [130.5  -3.4]
 [130.5  -3.2]
 [130.5  -3. ]
 [130.5  -2.8]
 [130.5  -2.6]
 [130.5  -2.4]
 [130.5  -2.2]]
(861, 2)
(861, 2)
[[-1.92825774 -3.04230502]
 [-1.93524991 -2.8902983 ]
 [-1.94188405 -2.7382749 ]
 [-1.94816006 -2.58623569]
 [-1.95407788 -2.43418156]
 [-1.95963743 -2.28211337]
 [-1.96483862 -2.13003199]
 [-1.96968141 -1.97793831]
 [-1.97416571 -1.82583321]
 [-1.97829149 -1.67371754]]
['-2.00E+0', '2.07E+0', '2.43E-2', '1.21E+0'] ['-3.04E+0', '3.04E+0', '-8.25E-18', '1.74E+0']


In [18]:
### take commands from patrick's amsr2_cfile.arts
ws.VectorCreate("antenna_reference_los")
ws.MatrixCreate("antenna_target_los")
ws.antenna_reference_los = np.array(los0)
ws.antenna_target_los = np.array(bsights, order="C")
ws.DiffZaAa(ws.antenna_dlos, ws.antenna_reference_los, ws.antenna_target_los)

# define sensor... done above, but apply gf4 typhon/arts object to antenna_response:
ws.sensor_norm = 1
ws.antenna_dim = 2
#ws.antenna_response.from_typhon?
ws.antenna_response = gf4 
print(ws.antenna_response.initialized)

ws.sensor_responseInit()
ws.sensor_responseAntenna()
#print(ws.sensor_pos.value)
ws.sensor_checkedCalc()


# should this get called? not yet -- will give bsights from designated positions, draw a line 
#ws.sensor_losGeometricFromSensorPosToOtherPositions(ws.target_pos=np.array([]))



True


In [19]:
# after performing all the checks, run yCalc:

ws.yCalc()

# y vector should be of size [nf*2 (stokes dim 2) ] * npix*nscans   ##formerly nfoot**2 

In [20]:

print(len(ws.y.value)/(nf*2))
for c in range(int(len(ws.y.value)/(nf*2))):
    print(ws.y.value[c*nf*2:(c+1)*nf*2])

40.0
[127.81197795  43.13759143 128.34399467  43.06563046 133.69235401
  42.22864806]
[127.81312334  43.13793165 128.34517398  43.06602711 133.69312238
  42.22832154]
[127.81410313  43.13840586 128.34618403  43.06656169 133.69337392
  42.22816308]
[127.81497383  43.139211   128.34708039  43.06743041 133.69344462
  42.22810291]
[127.81564176  43.13988518 128.34777243  43.06817144 133.69344462
  42.22810291]
[127.81610691  43.1404284  128.34826013  43.06878479 133.69337392
  42.22816308]
[127.81646141  43.14130098 128.34863254  43.06973054 133.69312238
  42.22832154]
[127.81663211  43.14228916 128.34881681  43.07079402 133.69235401
  42.22864806]
[127.80962193  43.13829415 128.34152768  43.06636602 133.69013251
  42.22934167]
[127.81060408  43.13879438 128.34254358  43.06692541 133.69008005
  42.2294435 ]
[127.81156674  43.13935658 128.34354188  43.06754841 133.69007822
  42.22949432]
[127.81246304  43.14019776 128.3444685   43.06845285 133.6900549
  42.22953052]
[127.81313112  43.140872

In [22]:
print(np.shape(ws.y_geo.value))
print('geo obs points for first scan: ')
print(ws.y_geo.value[:npix*nf*npol,3])  # columns are altitude, lat, lon, zenith, azimuth of measurement geoposition
print(ws.y_geo.value[:npix*nf*npol,4])  # note: zenith angles should be identical and be very close to EIA reported
# can compare [:,3] (az) to EIA from L1R (later), just 180-az to get EIA

(240, 5)
[125.09904978 125.09904978 125.09904978 125.09904978 125.09904978
 125.09904978 125.09904978 125.09904978 125.09904978 125.09904978
 125.09904978 125.09904978 125.09904978 125.09904978 125.09904978
 125.09904978 125.09904978 125.09904978 125.09904978 125.09904978
 125.09904978 125.09904978 125.09904978 125.09904978 125.09904978
 125.09904978 125.09904978 125.09904978 125.09904978 125.09904978
 125.09904978 125.09904978 125.09904978 125.09904978 125.09904978
 125.09904978 125.09904978 125.09904978 125.09904978 125.09904978
 125.09904978 125.09904978 125.09904978 125.09904978 125.09904978
 125.09904978 125.09904978 125.09904978]
[-2.18333653 -2.18333653 -2.18333653 -2.18333653 -2.18333653 -2.18333653
 -1.5878843  -1.5878843  -1.5878843  -1.5878843  -1.5878843  -1.5878843
 -0.99242905 -0.99242905 -0.99242905 -0.99242905 -0.99242905 -0.99242905
 -0.39697191 -0.39697191 -0.39697191 -0.39697191 -0.39697191 -0.39697191
  0.39697191  0.39697191  0.39697191  0.39697191  0.39697191  0.3